In [1]:
import os
import playsound
import speech_recognition as sr
import time
import sys
import ctypes
import wikipedia
import datetime
import json
import re
import webbrowser
import smtplib
import requests
import urllib
import urllib.request as urllib2
import time
import pyttsx3
from youtube_search import YoutubeSearch
import random
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
import pygame
import threading
from ctypes import *
from word2number import w2n
import wolframalpha

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
# init
path = ChromeDriverManager().install()
user32 = windll.user32
app_id = 'X9PRHQ-U333AQKRKH'
client = wolframalpha.Client(app_id) 

[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [C:\Users\My Laptop\.wdm\drivers\chromedriver\win32\87.0.4280.88\chromedriver.exe] found in cache


In [3]:
# speak out a string and print out skript
def speak(audio): 
    print("Jennie: {}".format(audio))
    engine = pyttsx3.init()
    voices = engine.getProperty('voices') 
    engine.setProperty('voice', voices[1].id)
    rate = engine.getProperty('rate')   
    engine.setProperty('rate', 145) 
    engine.say(audio)
    engine.runAndWait()

In [4]:
# use speech recognition library to listen to user and print out skript
def listen():
    robot_ear = sr.Recognizer()
    with sr.Microphone() as source:
        print("User: ", end='')
        audio = robot_ear.listen(source, phrase_time_limit=10)
    try:
        text = robot_ear.recognize_google(audio, language="vi")
        print(text)
        return text
    except:
        print("...")
        return 0

In [5]:
def listen2():
    robot_ear = sr.Recognizer()
    with sr.Microphone() as source:
        print("User: ", end='')
        audio = robot_ear.listen(source, phrase_time_limit=10)
    try:
        text = robot_ear.recognize_google(audio, language="vi")
        print(text)
        return text
    except:
        print("...")
        return ""

In [6]:
# get text (max 10 times) until something can be listened, change to lowercase
def get_text():
    for i in range(10):
        text = listen()
        if text:
            return text.lower()
        elif i < 9:
            speak("I can't really understand what you say.")
    time.sleep(2)
    stop()
    return 0

In [7]:
# get a number (max 5 times) until it can be listened
def get_num():
    for i in range(5):
        num = listen()
        if isinstance(num, str):
            if num.isnumeric():
                return int(num)
            elif i <= 4:
                if i == 4:
                    try:
                        return w2n.word_to_num(num)
                    except:
                        print('User: ...')
                else:
                    try:
                        return w2n.word_to_num(num)
                    except:
                        speak('I can not understand what you are saying. Please repeat it')
    time.sleep(2)
    speak('Sorry! I can not understand your request!')
    return 0

In [8]:
# say goodbye to different times
def stop():
    bye = ["Goodbye! See you later!", "Goodbye! Have a good day!", "See ya!"]
    bye2 = ["Goodbye! See you later!", "See ya!"]
    
    day_time = int(time.strftime('%H'))
    if 6 < day_time < 12:
        speak(random.choice(bye))
    elif 12 <= day_time < 18:
        speak(random.choice(bye2))
    elif 18 <= day_time < 22:
        speak("Goodbye. Have a good evening!")
    else:
        speak("Goodnight! See you tomorrow!")

In [9]:
# say hello to different times
def hello():
    day_time = int(time.strftime('%H'))
    if day_time < 12:
        speak("Good morning. Have a nice day.")
    elif 12 <= day_time < 18:
        speak("Good afternoon. Do you have any plan for this afternoon?.")
    else:
        speak("Good evening. Did you have dinner?")

In [10]:
# say name
def say_name():
    speak('Hi. I am Chung\'s personal assistant. You can call me Jennie.')

In [11]:
# tell about what Jennie can do
def help_me():
    speak("""I can help you with the following orders:
    1. Greetings and chatting
    2. Tell the time and date
    3. Open websites
    4. Google search
    5. Weather forcast
    6. Open videos on Youtube
    7. Set alarm
    8. Tell you about the world.""")

In [12]:
# tell time
def get_time():
    now = datetime.datetime.now()
    hour = now.hour
    minute = now.minute
    tail = 'AM'
    if hour > 12:
        hour -= 12
        tail = 'PM'
    speak('The time now is ' + str(hour) + ':' + str(minute) + ' ' + tail)

In [13]:
# tell date
def get_date():
    today = datetime.date.today()
    weekday = datetime.datetime.today().weekday() + 1
    day_dict = {1: 'Monday', 2: 'Tuesday',  
                3: 'Wednesday', 4: 'Thursday',  
                5: 'Friday', 6: 'Saturday', 
                7: 'Sunday'} 
    speak("Today is " + day_dict[weekday] + ', ' + today.strftime("%B %d, %Y"))

In [14]:
# forecast weather in a location
def current_weather():
    speak("Where do you want to see the weather forecast?")
    ow_url = "http://api.openweathermap.org/data/2.5/weather?"
    city = get_text()
    if not city:
        pass
    api_key = "fe8d8c65cf345889139d8e545f57819a"
    call_url = ow_url + "appid=" + api_key + "&q=" + city + "&units=metric"
    response = requests.get(call_url)
    data = response.json()
    if data["cod"] != "404":
        city_res = data["main"]
        current_temperature = city_res["temp"]
        current_pressure = city_res["pressure"]
        current_humidity = city_res["humidity"]
        wthr = data["weather"]
        weather_description = wthr[0]["description"]
        now = datetime.datetime.now()
        content = """
        Today is {day} {month} {year}.
        The temprature is {temp} grad.
        Humidity {humidity} procent.
        In general, the weather condition is {wd}.
        """.format(day = now.day, month = now.month, year= now.year, temp = current_temperature, humidity = current_humidity, wd = weather_description)
        speak(content)
        #time.sleep(10)
    else:
        speak("Can not find your address.")

In [15]:
# play youtube
def play_youtube():
    speak('Please choose a video name.')
    mysong = get_text()
    while True:
        result = YoutubeSearch(mysong, max_results=10).to_dict()
        if result:
            break
    url = 'https://www.youtube.com' + result[0]['url_suffix']
    webbrowser.open(url)
    speak("Your requested video is now opened.")

In [16]:
# google search, the searched term stands behind keyword kw
def google_search(text, kw):
    search_for = text.split(kw, 1)[1]
    try:
        speak('Okay!')
        driver = webdriver.Chrome(path)
        driver.get("http://www.google.com")
        que = driver.find_element_by_xpath("//input[@name='q']")
        que.send_keys(str(search_for))
        que.send_keys(Keys.RETURN)
    except:
        speak("I can not understand what you are saying. Please repeat it")

In [17]:
# open websites
def open_website(web_name):
    if web_name == 'sky':
        url = 'https://skyticket.sky.de/watch/sport/highlights'
    elif web_name == 'dazn':
        url = 'https://www.dazn.com/en-DE/home'
    elif web_name == 'facebook':
        url = 'https://www.facebook.com'
    elif web_name == 'gmail':
        url = 'https://accounts.google.com'
    elif web_name == 'google translate':
        url = 'https://translate.google.com.vn'
    elif web_name == 'google':
        url = 'https://www.google.com'
    elif web_name == 'lunar calendar':
        url = 'https://www.informatik.uni-leipzig.de/~duc/amlich/'
    elif web_name == 'live score':
        url = 'https://www.livescore.com/en/'
    elif web_name == 'youtube':
        url = 'https://www.youtube.com'
    elif web_name == 'wikipedia':
        url = 'https://www.wikipedia.org'
    
    webbrowser.open(url)
    speak(web_name + ' is now opened.')

In [18]:
# open popular news website
def news():
    webbrowser.open('https://vnexpress.net')
    webbrowser.open('https://kenh14.vn')
    webbrowser.open('https://www.bild.de')
    webbrowser.open('https://www.nytimes.com')
    speak('Popular news websites are now opened.')

In [19]:
# tell about something using wikipedia, the searched term stands behind keyword kw2
def tell_me_about(text, kw2):
    text = text.split(kw2, 1)[1]
    try:
        contents = wikipedia.summary(text, sentences=4) 
        #time.sleep(20)
        speak("According to wikipedia")
        speak(contents)

    except:
        speak("I can not understand what you are saying. Please repeat it")

In [20]:
# answer some knowledge question about the world, questions stands behind keyword kw5
def answer_questions(text, kw5):
    question = text  
    try: 
        res = client.query(question) 
        answer = next(res.results).text
        speak('The answer is: ' + answer)
    except:
        tell_me_about(text, kw5)

In [21]:
# play the alarm audio
def alarm_func():
    pygame.mixer.music.load('audios/alarm.mp3')
    pygame.mixer.music.play()

In [22]:
# set an alarm
def set_alarm():
    speak('In how many minutes do you want your alarm to be set?')
    minutes = get_num()
    if minutes:
        timer = threading.Timer(minutes*60, alarm_func)
        timer.start()
        speak("An alarm is set in " + str(minutes) + " minutes.")

In [23]:
# spell a word, the word to be spelled stand behind keyword kw4
def spell(text, kw4):
    spell_word = text.split(kw4, 1)[1]
    try:
        speak(spell_word + ' is spelled like:')
        speak(spell_word.replace('', ' ').upper())
    except:
        speak("I can not understand what you are saying. Please repeat it")

In [24]:
# Jennie - communicate and do requests function
def assistant():
    speak('Hello. My name is Jennie. How can i help you?')

    while True:
        text = listen2()
        text = text.lower()
        new_init = False
        global is_playing
        
        if ('jennie' not in text) and ('jenny' not in text) and ('annie' not in text) and ('randy' not in text) and ('gunny' not in text):
        # only do requests if there are 'Jennie' or some sound-like names in text    
            continue
        else:
            pygame.mixer.init()
            pygame.mixer.music.pause()
            
            speak('I am here! How can i help you?')
            text = get_text()
            if not text:
                break
            
            # spell a word
            elif ('spell' in text): 
                spell(text, 'spell ')
                
            # stop    
            elif ('shut down' in text) or ('bye' in text) or ('goodbye' in text) or ('goodnight' in text) or ('good night' in text) or ('rest' in text) or ('break' in text):
                stop()
                break
                
            # tell name
            elif ('your name' in text) or ('who are you' in text):
                say_name()
            
            # how are you?
            elif ('how are you' in text):
                speak('Yeah i\'m fine, thank you. And you?')
                
            # introduce functions
            elif ('what can you' in text):
                help_me()
            
            # hello
            elif ('hello' in text) or ('hi ' in text) or (text == 'hi'):
                hello()
                
            # ask time
            elif ('what time' in text) or ('the time' in text):
                get_time()
            
            # ask date
            elif ('today' in text):
                get_date()

            # google 
            elif ('google' in text) or ('google' in text):
                if ('search' in text): # google search
                    google_search(text, 'search ')
                elif ('translate' in text): # open google translate
                    open_website('google translate')
                else: # open google home page
                    open_website('google')
            
            # weather forecast
            elif ('weather' in text):
                current_weather()
            
            # open youtube website
            elif ('open youtube' in text):
                open_website('youtube')
            
            # play a video on youtube
            elif 'youtube' in text:
                play_youtube()
            
            # open lunar new year
            elif ('moon calendar' in text) or ('lunar calendar' in text):
                open_website('lunar calendar')
                
            # open football results
            elif ('live score' in text) or ('football score' in text) or ('football result' in text) or ('soccer result' in text) or ('soccer score' in text):
                open_website('live score')
                
            # open sport websites
            elif ('sport' in text) or ('soccer' in text) or ('football' in text):
                if ('sky' in text):
                    open_website('sky')
                else:
                    open_website('dazn')
            
            # open facebook
            elif ('facebook' in text):
                open_website('facebook')
                
            # open gmail
            elif ('mail' in text):
                open_website('gmail')
            
            # open wikipedia
            elif ('wiki' in text):
                open_website('wikipedia')
                
            # open news website
            elif ('news' in text) or ('new' in text) or ('newspaper' in text):
                news()
                
            # set alarm
            elif ('alarm' in text):
                set_alarm()

            # ask some knowledge questions who is, what is, where is...
            elif ('what is' in text) or ('who is' in text) or ('where is' in text):
                if 'what is' in text:
                    kw5 = 'what is '
                elif 'who is' in text:
                    kw5 = 'who is '
                elif 'where is' in text:
                    kw5 = 'where is '
                answer_questions(text, kw5)
            
            # listen about something using wikipedia
            elif ('tell me about' in text) or ('define' in text):
                if 'tell me about' in text:
                    kw2 = 'tell me about '
                elif 'define' in text:
                    kw2 = 'define '
                tell_me_about(text, kw2)
                
            else:
                speak("Sorry i can not understand your request!")
            
            pygame.mixer.music.unpause() 

In [25]:
assistant()

Jennie: Hello. My name is Jennie. How can i help you?
User: Jenny
Jennie: I am here! How can i help you?
User: Who are you
Jennie: Hi. I am Chung's personal assistant. You can call me Jennie.
User: ...
User: Jenny
Jennie: I am here! How can i help you?
User: How are you
Jennie: Yeah i'm fine, thank you. And you?
User: Jenny
Jennie: I am here! How can i help you?
User: Hello
Jennie: Good evening. Did you have dinner?
User: Jenny
Jennie: I am here! How can i help you?
User: What time is it
Jennie: The time now is 6:17 PM
User: Jenny
Jennie: I am here! How can i help you?
User: test today
Jennie: Today is Wednesday, January 13, 2021
User: ...
User: ...
User: Jenny
Jennie: I am here! How can i help you?
User: How do you spell computer
Jennie: computer is spelled like:
Jennie:  C O M P U T E R 
User: ...
User: ...
User: vannie
Jennie: I am here! How can i help you?
User: How is the weather
Jennie: Where do you want to see the weather forecast?
User: Hà Nội
Jennie: 
        Today is 13 1 202